In [1]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
torch.__version__

'0.4.1'

## Tensors

In [5]:
m = torch.rand(5,3)
m1 = torch.rand(5,3)
print(m)

tensor([[0.0421, 0.2183, 0.9574],
        [0.4908, 0.3341, 0.8571],
        [0.8419, 0.7114, 0.4916],
        [0.2660, 0.3869, 0.8245],
        [0.9793, 0.5440, 0.3444]])


In [4]:
print(m[:, 1])


 0.7309
 0.6608
 0.8538
 0.6203
 0.3185
[torch.FloatTensor of size 5]



In [20]:
m.size()[1:]

torch.Size([3])

In [6]:
torch.Tensor(m.numpy())

tensor([[0.0421, 0.2183, 0.9574],
        [0.4908, 0.3341, 0.8571],
        [0.8419, 0.7114, 0.4916],
        [0.2660, 0.3869, 0.8245],
        [0.9793, 0.5440, 0.3444]])

In [29]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [30]:
net = Net()

In [31]:
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [32]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0073, -0.0104,  0.0357,  0.0880, -0.0909,  0.1141,  0.0396, -0.0403,
          0.0571, -0.0429]], grad_fn=<ThAddmmBackward>)


In [33]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [34]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0615, grad_fn=<MseLossBackward>)


In [14]:
listNet = nn.Sequential(nn.Linear(10,5), nn.ReLU(), nn.Linear(5,1), nn.ReLU())

In [21]:
input = torch.autograd.Variable(torch.randn(2, 10))
output = listNet(input)
print(output)

tensor([[0.2366],
        [0.0548]], grad_fn=<ThresholdBackward0>)


In [22]:
input = torch.randn(2, 10)
output = listNet(input)
print(output)

tensor([[0.0475],
        [1.0172]], grad_fn=<ThresholdBackward0>)


In [35]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0055,  0.0095,  0.0018,  0.0025, -0.0032,  0.0087])


In [27]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [36]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update